# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import numpy as np
# Import the API key.
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [2]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "../Weather_Database/WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bluff,NZ,-46.6000,168.3333,9.89,92,97,1.96,overcast clouds
1,1,Juneau,US,58.3019,-134.4197,12.34,97,100,4.12,overcast clouds
2,2,Saint George,US,37.1041,-113.5841,21.55,31,0,2.57,clear sky
3,3,Punta Arenas,CL,-53.1500,-70.9167,9.06,49,40,10.29,scattered clouds
4,4,Geraldton,AU,-28.7667,114.6000,11.23,87,0,4.12,clear sky


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [3]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is your desired minimum temperature for your trip? "))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is your desired maximum temperature for your trip? "))

What is your desired minimum temperature for your trip? 5
What is your desired maximum temperature for your trip? 20


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [4]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) &
                                       (city_data_df["Max Temp"] <= max_temp)]
                                    
# Display sample data
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bluff,NZ,-46.6000,168.3333,9.89,92,97,1.96,overcast clouds
1,1,Juneau,US,58.3019,-134.4197,12.34,97,100,4.12,overcast clouds
3,3,Punta Arenas,CL,-53.1500,-70.9167,9.06,49,40,10.29,scattered clouds
4,4,Geraldton,AU,-28.7667,114.6000,11.23,87,0,4.12,clear sky
5,5,Kruisfontein,ZA,-34.0033,24.7314,18.82,83,92,4.64,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
682,682,Wurzen,DE,51.3667,12.7333,18.12,64,44,3.27,scattered clouds
684,684,La Ronge,CA,55.1001,-105.2842,9.14,71,40,3.09,scattered clouds
687,687,Charters Towers,AU,-20.1000,146.2667,19.48,94,42,3.71,scattered clouds
688,688,Sisimiut,GL,66.9395,-53.6735,5.31,59,100,3.20,overcast clouds


In [5]:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [6]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bluff,NZ,-46.6000,168.3333,9.89,92,97,1.96,overcast clouds
1,1,Juneau,US,58.3019,-134.4197,12.34,97,100,4.12,overcast clouds
3,3,Punta Arenas,CL,-53.1500,-70.9167,9.06,49,40,10.29,scattered clouds
4,4,Geraldton,AU,-28.7667,114.6000,11.23,87,0,4.12,clear sky
5,5,Kruisfontein,ZA,-34.0033,24.7314,18.82,83,92,4.64,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
682,682,Wurzen,DE,51.3667,12.7333,18.12,64,44,3.27,scattered clouds
684,684,La Ronge,CA,55.1001,-105.2842,9.14,71,40,3.09,scattered clouds
687,687,Charters Towers,AU,-20.1000,146.2667,19.48,94,42,3.71,scattered clouds
688,688,Sisimiut,GL,66.9395,-53.6735,5.31,59,100,3.20,overcast clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [7]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# Display sample data
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bluff,NZ,9.89,overcast clouds,-46.6000,168.3333,
1,Juneau,US,12.34,overcast clouds,58.3019,-134.4197,
3,Punta Arenas,CL,9.06,scattered clouds,-53.1500,-70.9167,
4,Geraldton,AU,11.23,clear sky,-28.7667,114.6000,
5,Kruisfontein,ZA,18.82,overcast clouds,-34.0033,24.7314,
6,Mataura,NZ,9.71,overcast clouds,-46.1927,168.8643,
8,Albany,US,17.19,overcast clouds,42.6001,-73.9662,
9,Jamestown,US,15.57,light rain,42.0970,-79.2353,
12,San Martin,AR,16.30,few clouds,-33.0810,-68.4681,
14,Akdagmadeni,TR,12.97,clear sky,39.6603,35.8836,


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [8]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE

## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [9]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [10]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# Display sample data
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bluff,NZ,9.89,overcast clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
1,Juneau,US,12.34,overcast clouds,58.3019,-134.4197,"Baranof Downtown, BW Signature Collection"
3,Punta Arenas,CL,9.06,scattered clouds,-53.1500,-70.9167,Hotel Hain
4,Geraldton,AU,11.23,clear sky,-28.7667,114.6000,Broadwater Mariner Resort
5,Kruisfontein,ZA,18.82,overcast clouds,-34.0033,24.7314,Oyster Bay House Rental


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [12]:
# Drop the rows where there is no Hotel Name.
c = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = c.dropna()

# Display sample data
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bluff,NZ,9.89,overcast clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
1,Juneau,US,12.34,overcast clouds,58.3019,-134.4197,"Baranof Downtown, BW Signature Collection"
3,Punta Arenas,CL,9.06,scattered clouds,-53.1500,-70.9167,Hotel Hain
4,Geraldton,AU,11.23,clear sky,-28.7667,114.6000,Broadwater Mariner Resort
5,Kruisfontein,ZA,18.82,overcast clouds,-34.0033,24.7314,Oyster Bay House Rental
...,...,...,...,...,...,...,...
682,Wurzen,DE,18.12,scattered clouds,51.3667,12.7333,Hotel Zur Post
684,La Ronge,CA,9.14,scattered clouds,55.1001,-105.2842,Northland Motor Hotel
687,Charters Towers,AU,19.48,scattered clouds,-20.1000,146.2667,Cattleman's Rest Motor Inn
688,Sisimiut,GL,5.31,overcast clouds,66.9395,-53.6735,Hotel Sisimiut


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [13]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [14]:
# Set the file name.
output_data_file = "weatherpy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [15]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [16]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>Bluff Homestead - Guesthouse & Campervan Park</dd>\n<dt>City</dt><dd>Bluff</dd>\n<dt>Country</dt><dd>NZ</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 9.89 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Baranof Downtown, BW Signature Collection</dd>\n<dt>City</dt><dd>Juneau</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 12.34 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hotel Hain</dd>\n<dt>City</dt><dd>Punta Arenas</dd>\n<dt>Country</dt><dd>CL</dd>\n<dt>Current Weather</dt><dd>scattered clouds and 9.06 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Broadwater Mariner Resort</dd>\n<dt>City</dt><dd>Geraldton</dd>\n<dt>Country</dt><dd>AU</dd>\n<dt>Current Weather</dt><dd>clear sky and 11.23 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Oyster Bay House Rental</dd>\n<dt>City</dt><dd>Kruisfontein</dd>\n<dt>Country</dt><dd>ZA</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 18.82 °F</dd>\

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [17]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
0,-46.6000,168.3333
1,58.3019,-134.4197
3,-53.1500,-70.9167
4,-28.7667,114.6000
5,-34.0033,24.7314
6,-46.1927,168.8643
9,42.0970,-79.2353
12,-33.0810,-68.4681
14,39.6603,35.8836
18,62.4560,-114.3525


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [18]:
# Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Create a figure to add the Google map as a layer
fig.add_layer(markers)

# Display the figure containing the map
fig

Figure(layout=FigureLayout(height='420px'))